In [335]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, auc
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [20]:
pd.set_option( "display.max_columns", None)

In [21]:
df = pd.read_csv('supermarket_sales - Sheet1.csv')

In [22]:
df

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1


Tento dataset obsahuje data o nákupech zákazníků v různých městech a odvětví.
Dataset obsahuje tyto atributy:

#####  Gender: Pohlaví zákazníka. Důležitý atribut, který budu zkoumat.

City: lokalita obchodu

Customer type: Typ zákazníka, člen pokud použil zákaznickou kartu nebo normální zákazník bez karty 




Product line: Obecné skupiny kategorií položek - Elektronické příslušenství, Módní doplňky, Potraviny a nápoje, Zdraví a krása, Domácnost a styl života, Sport a cestování.

Unit price: Cena každého produktu v $

Quantity: Počet produktů zakoupených zákazníkem

Tax: Daňový poplatek ve výši 5% pro zákazníka, který nakupuje

Total: Celková cena včetně daně

Date: Datum nákupu (záznamy jsou k dispozici od ledna 2019 do března 2019)

Time: Čas nákupu (od 10:00 do 21:00)

Payment: Platba použitá zákazníkem při nákupu (jsou k dispozici 3 metody - hotovost, kreditní karta a elektronická peněženka)

COGS: Náklady na prodané zboží
Gross margin percentage: Gross margin percentage

Gross income: Procento hrubé marže

Rating: Hrubý příjem





In [23]:
df.describe()

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


In [24]:
df.describe(exclude=np.number)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Date,Time,Payment
count,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,1000,3,3,2,2,6,89,506,3
top,750-67-8428,A,Yangon,Member,Female,Fashion accessories,2/7/2019,19:48,Ewallet
freq,1,340,340,501,501,178,20,7,345


In [25]:
df.describe(include=np.number)

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


In [26]:
df.apply(lambda x: x.isna().sum()).sort_values(ascending=False)

Invoice ID                 0
Total                      0
gross income               0
gross margin percentage    0
cogs                       0
Payment                    0
Time                       0
Date                       0
Tax 5%                     0
Branch                     0
Quantity                   0
Unit price                 0
Product line               0
Gender                     0
Customer type              0
City                       0
Rating                     0
dtype: int64

Dataset neobsahuje žádné prázdné řádky/sloupce, takže nebudeme muset nic dropovat.

In [27]:
df.Gender.value_counts()

Female    501
Male      499
Name: Gender, dtype: int64

O našem datasetu se dá říct, že vyvážený. Protože, námi hledaný atribut je v skoro poměru 1:1.

In [28]:
df = df.loc[:, ['Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating']].copy()

Vyberu z tadasetu jen potřebné sloupce

In [29]:
df.Branch.value_counts()

A    340
B    332
C    328
Name: Branch, dtype: int64

In [36]:
pay = ['Cash',  'Credit card', 'Ewallet']
tr = OrdinalEncoder(categories = [pay])
df['Payment'] = tr.fit_transform(df[['Payment']])

Jen u atributu Payment mohu rozeznat pořadí proto je zaměním za 0 1 2, u zbytku nečíselných hodnot použiju One Hot Encoding

In [45]:
df['Gender']=np.where(df['Gender']=='Female',0,1)

Gender zaměním za 0 a 1, jelikož ho chci použít ke kasifikaci

In [46]:
dummies = pd.get_dummies(df)
dummies

,Gender,Unit price,Quantity,Tax 5%,Total,Payment,cogs,gross margin percentage,gross income,Rating,Branch_A,Branch_B,Branch_C,City_Mandalay,City_Naypyitaw,City_Yangon,Customer type_Member,Customer type_Normal,Product line_Electronic accessories,Product line_Fashion accessories,Product line_Food and beverages,Product line_Health and beauty,Product line_Home and lifestyle,Product line_Sports and travel
0,0,74.69,7,26.1415,548.9715,2.0,522.83,4.761905,26.1415,9.1,1,0,0,0,0,1,1,0,0,0,0,1,0,0
1,0,15.28,5,3.8200,80.2200,0.0,76.40,4.761905,3.8200,9.6,0,0,1,0,1,0,0,1,1,0,0,0,0,0
2,1,46.33,7,16.2155,340.5255,1.0,324.31,4.761905,16.2155,7.4,1,0,0,0,0,1,0,1,0,0,0,0,1,0
3,1,58.22,8,23.2880,489.0480,2.0,465.76,4.761905,23.2880,8.4,1,0,0,0,0,1,1,0,0,0,0,1,0,0
4,1,86.31,7,30.2085,634.3785,2.0,604.17,4.761905,30.2085,5.3,1,0,0,0,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,40.35,1,2.0175,42.3675,2.0,40.35,4.761905,2.0175,6.2,0,0,1,0,1,0,0,1,0,0,0,1,0,0
996,0,97.38,10,48.6900,1022.4900,2.0,973.80,4.761905,48.6900,4.4,0,1,0,1,0,0,0,1,0,0,0,0,1,0
997,1,31.84,1,1.5920,33.4320,0.0,31.84,4.761905,1.5920,7.7,1,0,0,0,0,1,1,0,0,0,1,0,0,0
998,1,65.82,1,3.2910,69.1110,0.0,65.82,4.761905,3.2910,4.1,1,0,0,0,0,1,0,1,0,0,0,0,1,0


Všechny zbylé ne numerické atributy se musí zakódovat na numerické pomocí funkce 'pd.get_dummies', aby s nimi mohl následně klasifikační algoritmus pracovat 

In [47]:
X, y = dummies.loc[:, (dummies.columns != 'Gender') ], dummies.loc[:, 'Gender']
X.shape, y.shape

((1000, 23), (1000,))

Tento dataset si rozdělím na daný sloupec, který zkoumám (v mojem případě Gender) a všechny ostatní.

In [319]:
best_scores = []
best = 0
best_index = 0

    
skf = StratifiedKFold(n_splits=41, shuffle = True, random_state = 10)
scores = list()
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf = DecisionTreeClassifier(splitter = "best", criterion = "gini", max_depth = 5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    scores.append(f1_score(y_test, y_pred))

    if best < np.mean(scores):
        best_scores = scores
        best = np.mean(scores)

print(f'Průměrné skóre je {np.mean(best_scores)}, Min: {np.min(best_scores)}, Max: {np.max(best_scores)} při algoritmu'
     f' DecisionTreeClassifier  ')
#np.mean(best_scores), np.min(best_scores), np.max(best_scores), best_index

Průměrné skóre je 0.6305034297420147, Min: 0.4444444444444445, Max: 0.7741935483870968 při algoritmu DecisionTreeClassifier  


Při klasifikaci  pomocí klasifikátoru DecisionTreeClassifier jsem dosáhl nejlepší predikce pohlaví 
63% s použitím největší hloubky 5. Použil jsem 41 splitů, což je celkem hodně, protože jde vidět, že se liší nejhorší a nejlepší výsledek o přibližně 
33 %. 

In [332]:
result = 0
result_score = 0
index_inner = 0

scores_field = []
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 10)

for i in range (1, 100):
    scores = list()
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = RandomForestClassifier(n_estimators=i, random_state = 10,max_depth = 7)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scores.append(f1_score(y_test, y_pred))

    scores
    np.mean(scores), np.min(scores), np.max(scores)
    scores_field.append(scores)
    if result < np.mean(scores):
        result = np.mean(scores)
        result_score = scores
        index_inner = i
     
np.mean(result_score), np.min(result_score), np.max(result_score), index_inner 
print (f'Průměrné skóre je {np.mean(result_score)}, Min: {np.min(result_score)}, Max: {np.max(result_score)}, '
       f' počtem estimátorů {index_inner}' )

Průměrné skóre je 0.5563251228596534, Min: 0.5209302325581395, Max: 0.5979381443298969,  počtem estimátorů 40


Při klasifikaci pomocí klasifikátoru RandomForestClassifier a maximimální hloubky 7 uzlů, jsem dosáhl nejlepší predikce pohlaví 55,6 % s počtem estimátorů 40. 

In [334]:
skf = StratifiedKFold(n_splits=8, shuffle=True, random_state = 10)
scores = list()
activ = [ 'relu', 'identity','logistic', 'tanh' ]#, 'logistic', 'tanh' ,'relu''identity','relu',
solve = ['sgd', 'adam']#, 'sgd', 'adam'
result = []
for acti in activ:
    
    for solv in solve:
        scores = list()
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            clf = MLPClassifier(hidden_layer_sizes=(20,5), activation=acti, solver=solv, max_iter=100000, random_state=13)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            scores.append(f1_score(y_test, y_pred))
        if not result:
            result = [acti, solv, np.mean(scores), np.min(scores), np.max(scores)]
        elif result[2] <  np.mean(scores):
            result = [acti, solv, np.mean(scores), np.min(scores), np.max(scores)]
        #print(acti, solv, np.mean(scores), np.min(scores), np.max(scores))
#scores
print(result)

['relu', 'sgd', 0.6657682899078393, 0.6631016042780749, 0.6702127659574468]


Při klasifikaci pomocí MLPClassifier jsem dosáhl nejlepšího průměrného skóre 66.6 % s 
parametry solver = sgd a activation = relu

In [370]:
skf = StratifiedKFold(n_splits=17, shuffle=True, random_state = 10)
scores = list()
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
weights = ['uniform', 'distance']
result = []
for alg in algorithm:
    
    for wei in weights:
        scores = list()
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            clf = KNeighborsClassifier(n_neighbors=5,weights = wei,algorithm = alg )
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            scores.append(f1_score(y_test, y_pred))
        if not result:
            result = [alg, wei, np.mean(scores), np.min(scores), np.max(scores)]
        elif result[2] <  np.mean(scores):
            result = [alg, wei, np.mean(scores), np.min(scores), np.max(scores)]
       
print(result)

['auto', 'distance', 0.5504655205355704, 0.4285714285714286, 0.6206896551724138]


Při klasifikaci pomocí KNeighborsClassifier jsem dosáhl nejlepšího průměrného skóre pouze 55.0 % s 
parametry algorithm = auto a weights = distance

Závěrem bych dodal, že nejvíce dosažené úspěšnosti odhadu pohlaví nakupujícího v tomle projektu je průměrně 66.6 % pomocí klasifikátoru MLPClassifier. Úspěšnost skoro 7 lidí z deseti mi nepřijde nějak převratná, ale vzhledem k vyváženosti datasetu a výsledků ostatních klasifikátorů, je tenhle výsledek ještě solidní. Pro výpočet úspěšnosti jsem použil f1_score, která bere v potaz true positive, true negative, false positive a false negative.